In [21]:
import sqlite3
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.manifold import TSNE


conn = sqlite3.connect('../cultura_1.db')
cursor = conn.cursor()

df_object = pd.read_sql_query("SELECT * FROM created_work", conn)


In [30]:
df_count= df_object[['instance_label', 'super_instance_label']].drop_duplicates()

In [38]:
df_model = df_count[['super_instance_label']].dropna().drop_duplicates().reset_index(drop=True)

In [39]:
model = SentenceTransformer('all-mpnet-base-v2')

In [40]:
embeddings = model.encode(df_model['super_instance_label'].to_list(), show_progress_bar=True)

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

In [41]:
reducer = TSNE(n_components=1)
reduced_embeddings = reducer.fit_transform(embeddings)

In [42]:
df_model['rank'] = reduced_embeddings
df_model = df_model.sort_values('rank')

In [46]:
df_final = pd.merge(df_model, df_object, on = 'super_instance_label', how = 'outer')
df_final = df_final.sort_values('rank', ascending=False)

In [73]:
#df_final[df_final['super_instance_label'] == np.nan]

In [75]:
res = df_final.groupby('instance_label')['super_instance_label'].apply(list).reset_index()

In [76]:
res['super_instance_label'] = res['super_instance_label'].apply(lambda x : list(set(x)))
#res['super_instance_label'] = res['super_instance_label'].apply(lambda x : ' | '.join(x) if x is not None else None)

In [49]:
df_final.to_clipboard(index=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [79]:
df_object = pd.read_sql_query("SELECT * FROM individual_created_work", conn)


In [ ]:
df_object = df_object[['individual_wikidata_id', 'notable_work_wikidata_id', 'creation_year', 'instance_label']].drop_duplicates()
df_object = df_object.drop_duplicates('notable_work_wikidata_id', keep= 'first')

df_individual = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_individual = df_individual[['individual_wikidata_id', 'birthyear']]

df_individual_region = pd.read_sql_query("SELECT * FROM individuals_regions", conn)
df_individual_region = df_individual_region[['individual_wikidata_id', 'region_name']].drop_duplicates()

df = pd.merge(df_object, df_individual, on = 'individual_wikidata_id')
df['productive_year'] = df['birthyear'] + 35
df['year'] = df.apply(lambda row: row['creation_year'] if row['creation_year'] is not None else row['productive_year'], axis=1)
df = df.dropna(subset= 'year')
#df = df[df['year']<=1860]

df = pd.merge(df, df_individual_region, on = 'individual_wikidata_id')